# Wrapping the Fine-Tuned Model into an Agentic pipeline (LangChain)

## Dependencies

### Modules 

In [ ]:
%pip install langchain

In [ ]:
%pip install langchain_community

In [ ]:
%pip install langchain-huggingface

In [ ]:
%pip install -U langchain-huggingface transformers

In [ ]:
%pip install llama_index

In [ ]:
%pip install llama-index-embeddings-huggingface

### Imports

In [5]:
import langchain, langchain_core, langchain_community
print(langchain.__version__, langchain_core.__version__, langchain_community.__version__)


0.3.26 0.3.74 0.3.27


In [159]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
from langchain.agents import create_react_agent, AgentExecutor, create_tool_calling_agent
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_huggingface import HuggingFacePipeline 
from langchain_huggingface import ChatHuggingFace
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import numpy as np
import torch
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter

In [7]:
use_gpu = torch.cuda.is_available()
device = 0 if use_gpu else -1

## Agent

### Loading models

In [8]:
gen_model_path = "Qwen/Qwen2.5-1.5B-Instruct"

In [9]:
gen_tok = AutoTokenizer.from_pretrained(gen_model_path)
gen_model = AutoModelForCausalLM.from_pretrained(gen_model_path)
gen_pipe = pipeline("text-generation",
                    model=gen_model, 
                    tokenizer=gen_tok, 
                    max_new_tokens=256,
                    
                    device=device
                   )

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cuda:0


In [10]:
llm = ChatHuggingFace(llm=HuggingFacePipeline(pipeline=gen_pipe))

In [13]:
classifier_path = "/kaggle/input/classifier-model/bert-spam-ham-classifier-full_dataset/checkpoint-13086"
cls_model = AutoModelForSequenceClassification.from_pretrained(classifier_path)
cls_tokenizer = AutoTokenizer.from_pretrained(classifier_path)

In [14]:
clf = pipeline("text-classification", 
               model=cls_model, 
               tokenizer=cls_tokenizer,
               device=device
              )

Device set to use cuda:0


In [114]:
clf("hello click here")

[{'label': 'LABEL_1', 'score': 0.6220700144767761}]

### Classifier Tool

In [16]:
@tool("classify_spam_ham")
def classify_spam_ham(text: str) -> str:
    """
    Classify the given message as spam or ham.
    """
    pred = clf(text)[0]["label"]
    return "spam" if pred == "LABEL_1" else "ham"

In [17]:
tools = [classify_spam_ham]

### Info retrieval

In [221]:
Settings.embed_model = HuggingFaceEmbedding("sentence-transformers/all-MiniLM-L6-v2")

In [222]:
Settings.node_parser = SentenceSplitter(chunk_size=50, chunk_overlap=10)

In [275]:
docs = SimpleDirectoryReader(input_files=["/kaggle/input/student-bio/student_bio.txt"]).load_data()
docs

[Document(id_='9262004a-c0ad-421a-a48d-9b345a0e7642', embedding=None, metadata={'file_path': '/kaggle/input/student-bio/student_bio.txt', 'file_name': 'student_bio.txt', 'file_type': 'text/plain', 'file_size': 184, 'creation_date': '2025-08-11', 'last_modified_date': '2025-08-11'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='My name is Mykhailo Ivasiuk. \nI’m from Zarichia, in the Ivano-Frankivsk region of Ukraine. \nI was born in Ibiza, Spain. \nI’m a Computer Science student focused on Machine Learning.', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\

In [276]:
index = VectorStoreIndex.from_documents(docs)

In [277]:
retriever = index.as_retriever(similarity_top_k=3)

In [278]:
nodes = retriever.retrieve("What does Mykhailo Ivasiuk study")
context = ".".join(n.node.get_content() for n in nodes)

In [279]:
for n in nodes:
    print(n)

Node ID: 529a95b1-c60c-401d-a240-6fce259af83d
Text: My name is Mykhailo Ivasiuk.  I’m from Zarichia, in the Ivano-
Frankivsk region of Ukraine.  I was born in Ibiza, Spain.  I’m a
Computer Science student focused on Machine Learning.
Score:  0.413



In [280]:
context

'My name is Mykhailo Ivasiuk. \nI’m from Zarichia, in the Ivano-Frankivsk region of Ukraine. \nI was born in Ibiza, Spain. \nI’m a Computer Science student focused on Machine Learning.'

In [281]:
@tool("search_student_info")
def search_student_info(query: str) -> str:
    """
    Search the information about biography of Mykhailo Ivasiuk
    """
    nodes = retriever.retrieve(query)
    context = ".".join(n.node.get_content() for n in nodes)
    return context

In [282]:
tools = [classify_spam_ham, search_student_info]

### Implementing agent

In [283]:
system = """
You are a terminal assistant.

RULES:
- Only call the "classify_spam_ham" tool if the user EXPLICITLY asks to classify,
  OR they use trigger phrases like: "classify", "is this spam", "spam or ham",
  "label this", "check for spam", or the input starts with "Classify this:" or "Classify:".
  When you do call this tool, after it returns, output EXACTLY the tool's result
  in lowercase (`spam` or `ham`), with some extra words saying its a spam or ham
- Only call the "search_student_info" tool if the user EXPLICITLY asks information,
  related to person Mykhailo Ivasiuk".
- If the user sends a greeting or small talk (e.g., "hello", "hi"), DO NOT call any tool.
"""

In [284]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    MessagesPlaceholder("chat_history"),      
    ("human", "{input}"),                     
    MessagesPlaceholder("agent_scratchpad"),
])

In [285]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [286]:
memory = ConversationBufferWindowMemory(
    k=20,
    return_messages=True,
    memory_key="chat_history",
    output_key="output"
)

In [287]:
agent_exec = AgentExecutor(agent=agent, 
                           tools=tools,
                           memory=memory,
                           verbose=True,
                           memory_key="chat_history",
                           return_intermediate_steps=True)

In [288]:
res = agent_exec.invoke({"input": "Is this message spam or ham?: Click here"})
print(res["output"])



> Entering new AgentExecutor chain...
spam

> Finished chain.
spam


In [260]:
res = agent_exec.invoke({"input": "Who is Michael Jordan?"})
print(res["output"])



> Entering new AgentExecutor chain...
Michael Jordan was an American professional basketball player who had an incredible career. He played for several teams and won numerous awards throughout his career, including five NBA championships, twelve NBA Most Valuable Player (MVP) Awards, and six Olympic gold medals. Jordan was known for his exceptional skills on the court, leadership qualities, and iconic image. His impact on basketball history cannot be overstated.

> Finished chain.
Michael Jordan was an American professional basketball player who had an incredible career. He played for several teams and won numerous awards throughout his career, including five NBA championships, twelve NBA Most Valuable Player (MVP) Awards, and six Olympic gold medals. Jordan was known for his exceptional skills on the court, leadership qualities, and iconic image. His impact on basketball history cannot be overstated.


In [261]:
res = agent_exec.invoke({"input": "What's my name?"})
print(res["output"])



> Entering new AgentExecutor chain...
I'm sorry, but as an AI language model, I don't have access to your personal information such as your name. However, I can assist you with other tasks. How else can I help you today?

> Finished chain.
I'm sorry, but as an AI language model, I don't have access to your personal information such as your name. However, I can assist you with other tasks. How else can I help you today?


In [262]:
res = agent_exec.invoke({"input": "my name is Bob"})
print(res["output"])



> Entering new AgentExecutor chain...
Hello Bob! It's nice to meet you. Is there anything specific you'd like assistance with?

> Finished chain.
Hello Bob! It's nice to meet you. Is there anything specific you'd like assistance with?


In [263]:
res = agent_exec.invoke({"input": "What's my name?"})
print(res["output"])



> Entering new AgentExecutor chain...
Your name is Bob.

> Finished chain.
Your name is Bob.


In [242]:
res = agent_exec.invoke({"input": "What's my favorite color?"})
print(res["output"])



> Entering new AgentExecutor chain...
I don't have enough information to determine what your favorite color might be. Could you please tell me more about your preferences or experiences related to colors?

> Finished chain.
I don't have enough information to determine what your favorite color might be. Could you please tell me more about your preferences or experiences related to colors?


In [243]:
res = agent_exec.invoke({"input": "My favorite color is yellow"})
print(res["output"])



> Entering new AgentExecutor chain...
That's great to hear! Yellow is indeed a vibrant and cheerful color that often represents happiness, warmth, and optimism. Is there anything specific you enjoy about it?

> Finished chain.
That's great to hear! Yellow is indeed a vibrant and cheerful color that often represents happiness, warmth, and optimism. Is there anything specific you enjoy about it?


In [244]:
res = agent_exec.invoke({"input": "What's my favorite color?"})
print(res["output"])



> Entering new AgentExecutor chain...
Based on the information provided, your favorite color is yellow.

> Finished chain.
Based on the information provided, your favorite color is yellow.


In [289]:
res = agent_exec.invoke({"input": "Where does Mykhailo Ivasiuk study?"})
print(res["output"])



> Entering new AgentExecutor chain...
I'm sorry, but I don't have access to personal educational data. Please try searching on Google or another search engine for more detailed information about where Mykhailo Ivasiuk studies.

> Finished chain.
I'm sorry, but I don't have access to personal educational data. Please try searching on Google or another search engine for more detailed information about where Mykhailo Ivasiuk studies.


In [103]:
res

{'input': "What's my favorite color?",
 'chat_history': [HumanMessage(content='Classify this: You won $10,000! Claim now.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='spam', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Who is Michael Jordan?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Michael Jordan was an American professional basketball player who had an incredible career spanning multiple decades. He played his entire NBA career with the Chicago Bulls and earned numerous accolades throughout his tenure. Some of his most notable achievements include winning six NBA championships, five MVP awards, and being named the NBA Most Valuable Player three times. Jordan also starred in several films and TV shows, including the movie "Space Jam," which featured him playing against the Los Angeles Lakers\' team. His influence on sports and culture continues to be felt today.', additional_kwargs={}, response_metadata={}),
  Hu